In [10]:
######  CODE WAAR GELUIDSINDICATOREN OPGESLAGEN ZIT IN NUMPY ARRAYS - INPUT(recorder_file, EB_file (optioneel), NN_model)
from tensorflow.keras.models import load_model
import numpy as np
from pandas import read_csv
from geopandas import GeoDataFrame
import pickle
import os
import re
import warnings
from shapely.geometry import Point, LineString, Polygon
from shapely.affinity import rotate
from shapely.ops import transform
from shapely.wkt import loads
from shapely import wkb
import pyproj
import osmnx as ox
import matplotlib.pyplot as plt
from copy import copy
import warnings
import time

import ml_indicators_EB2 as mli
from GroupScaler import GroupScaler

ox.config(timeout=1000)

file_receivers = '../../../../Receiver/RECEIVERSAlpine.csv'
file_indicators_save = '../../../../Predictions/alpineML4.npy'
file_geom_save = '../../../../Predictions/alpineMLgeom4.npy'
file_features_save = '../../../../Predictions/alpineMLfeatures4.npy'
read_EB = True
file_EB = '../../../../Roads/DataCityEB/alpine_streets_drive.csv'
file_model = 'CNN4values_new140epochs'
file_input_scaler = 'CustomGroupInputScaler.pkl'
file_output_scaler = 'CustomOutputScalerPBF2.pkl'
point_in_Lambert = False
building_nearby = False

#srid = 31370        #FLANDERS
#srid = 2062         #BARCELONA
srid = 31287        #AUSTRIA

file = read_csv(file_receivers)
file['geometry'] = file['geometry'].apply(loads)
file = GeoDataFrame(file, geometry='geometry')
data = file['geometry']

model = load_model(file_model)

scaler_input = pickle.load(open(file_input_scaler,'rb'))
scaler_output = pickle.load(open(file_output_scaler,'rb'))

epsg_4326 = pyproj.CRS('EPSG:4326')
epsg_target = pyproj.CRS("EPSG:%d" % srid)

project_4326_target = pyproj.Transformer.from_crs(epsg_4326, epsg_target, always_xy=True).transform
project_target_4326 = pyproj.Transformer.from_crs(epsg_target, epsg_4326, always_xy=True).transform

if read_EB:
    df_roadsEB = read_csv(file_EB)
    df_roadsEB['geometry'] = df_roadsEB['geometry'].apply(loads)
    for i in range(len(df_roadsEB)):
        if str(df_roadsEB['lanes'][i])[0] == '[':
            df_roadsEB['lanes'][i] = min(list(map(int, eval(df_roadsEB['lanes'][i]))))
    df_roadsEB = GeoDataFrame(df_roadsEB, geometry='geometry', crs='EPSG:4326')
    df_roadsEB = df_roadsEB.to_crs("EPSG:%d" % srid)
    
def test_if_in_building(x_point, y_point, direction = -1):
#    try:
    df_buildings = ox.geometries_from_point((y_point, x_point), dist=25, tags={"building": True})
#    except:
#        print("finding buildings took too long")
    df_buildings = df_buildings.to_crs("EPSG:%d" % srid)
    pointinbuild = []
    newpointlist = []
    if len(df_buildings.index) == 0:
        newpointlist.append(Point(x_point, y_point))
        return newpointlist
    point_target: Point = transform(project_4326_target, Point(x_point, y_point))
    for key, geom in df_buildings["geometry"].items():
        ptinbuild = point_target.within(geom)
        pointinbuild.append(ptinbuild)
    if not any(pointinbuild):
        newpointlist.append(Point(x_point, y_point))
        return newpointlist
    else:
        print("  Point in building (moving point...)")
        for i in range(4):
            x_point = point_target.x
            y_point = point_target.y
            if i==0 and (direction == -1 or direction == 0):
                x_point += 5
            elif i==1 and (direction == -1 or direction == 1):
                x_point -= 5
            elif i==2 and (direction == -1 or direction == 2):
                y_point += 5
            elif i==3 and (direction == -1 or direction == 3):
                y_point -= 5
            else:
                continue
            point_osm: Point = transform(project_target_4326, Point(x_point, y_point))
            x_point = point_osm.x
            y_point = point_osm.y
            newpointlist.extend(test_if_in_building(x_point, y_point, direction=i))
        return newpointlist

max_osm_dist = 2000
close_distance = 200
building_distance = 20
nb_angles = 36

input_data = np.array([]).reshape(0, 452)
output_geom = []
i = 0

for receiver_point in data:
    with warnings.catch_warnings(record=True):
        start1 = time.time()
        print("READING POINT (%d/%d)" % (i, len(data)))
        if point_in_Lambert:
            osm_point: Point = transform(project_target_4326, receiver_point)
        else:
            osm_point: Point = receiver_point
            receiver_point: Point = transform(project_4326_target, receiver_point)
        receiver_point = wkb.loads(wkb.dumps(receiver_point, output_dimension=2))

        x_point = osm_point.x
        y_point = osm_point.y
        pointlist = test_if_in_building(x_point, y_point)
        early_stop = False
        for subpoint in pointlist:
            if early_stop:
                continue #once it was decided for one of the shifted points that calculation is useless, we stop all
            x_point = subpoint.x
            y_point = subpoint.y
            receiver_point: Point = transform(project_4326_target, Point(x_point, y_point))

            start2 = time.time()
            if read_EB:
                polygon = receiver_point.buffer(max_osm_dist) #is this correct
                df_roads = df_roadsEB.loc[df_roadsEB.intersects(polygon)].reset_index()
            else:
                df_roads = ox.geometries_from_point((y_point, x_point), dist=max_osm_dist, tags={"highway": True})
                df_roads = df_roads.to_crs("EPSG:%d" % srid)
            print('  Load roads: ', time.time() - start2)
            if len(df_roads.index) == 0:
                print("  No roads")
                #i += 1
                early_stop = True # when there are no roads within 2 km we skip calculations
                continue

            start2 = time.time()
#        try:
            df_buildings = ox.geometries_from_point((y_point, x_point), dist=max_osm_dist, tags={"building": True})
#        except:
#            print("finding buildings took too long")
            df_buildings = df_buildings.to_crs("EPSG:%d" % srid)
            print('  Load buildings: ', time.time() - start2)

            if building_nearby:
                polygon = receiver_point.buffer(building_distance)
                df_nearby_building = df_buildings.loc[df_buildings.intersects(polygon)]
                if len(df_nearby_building.index) == 0:
                    print("  No building nearby")
                    #i += 1
                    early_stop = True #if there is no building within 200m it is not useful to calculate
                    continue 

            traffic = []
            start2 = time.time()
            for j in range(len(df_roads)):
                traffic.append(mli.get_traffic_intensity(df_roads, j))
            df_roads["lv"] = traffic
            print('  Calculate traffic intensity: ', time.time() - start2)
            polygon = receiver_point.buffer(close_distance)
            df_close_roads = df_roads.loc[df_roads.intersects(polygon)]
            output_array = []
            if len(df_close_roads.index) == 0:
                print("  No close roads")
                output_array = np.zeros((24,432))
                #i += 1
                #continue # why would we stop and ignore far away roads?
            else:
                df_close_buildings = df_buildings.loc[df_buildings.intersects(polygon)]
        # Building densities by angle
                output_array += list(mli.get_building_surface_by_angle(receiver_point, close_distance, nb_angles, df_close_buildings).values())
        # Closest building by angle
                build_in_dist = list(mli.get_closest_object_distance_by_angle(receiver_point, close_distance, nb_angles, df_close_buildings).values())
                new_build_in_dist = [1 / (a+1) for a in build_in_dist]
                for b in range(len(new_build_in_dist)):
                    if new_build_in_dist[b] == -1:
                        new_build_in_dist[b] = 0
                output_array += new_build_in_dist
                output_array = np.tile(output_array, (24, 1))
        # Closest roads with traffic intensities between min_lv - max_lv by angle
                min_lv = 0
                start2 = time.time()
                stack = np.stack(df_close_roads["lv"])
                for max_lv in [10, 20, 40, 80, 160, 320, 640, 1280, 2560, 99999]:
                    sub_output_array = np.zeros((24, 36))
                    for hour in range(24):
                        maskindex = stack[:, hour]
                        mask = (maskindex >= min_lv) & (maskindex < max_lv)
                        filtered_lv_roads = df_close_roads.loc[mask]#[(df_close_roads["lv"][hour] > min_lv) & (df_close_roads["lv"][hour] <= max_lv)]
                # if len(filtered_lv_roads) == 0
                # output_array += [-1]
                        road_in_dist = list(mli.get_closest_object_distance_by_angle(receiver_point, close_distance, nb_angles, filtered_lv_roads).values())
                        new_road_in_dist = [1 / (x+1) for x in road_in_dist]
                        for k in range(len(new_road_in_dist)):
                            if new_road_in_dist[k] == -1:
                                new_road_in_dist[k] = 0
                        sub_output_array[int(hour)] = new_road_in_dist
                    output_array = np.concatenate((output_array, sub_output_array), axis=1)
                    min_lv = max_lv
        # Adding of total building surface between distances & total road length of certain traffic intensities between the distances
            print(output_array.shape) #debug
            print("  Calculate close objects: ", time.time() - start2)
            start2 = time.time()
            stack = np.stack(df_roads["lv"])
            min_dist = 0
            for max_dist in [200, 400, 800, 1600]:
                output_array = np.c_[output_array, np.ones(24)*mli.get_building_surface_between_distances(receiver_point, min_dist, max_dist, df_buildings)]
                min_lv = 200
                for max_lv in [400, 800, 1600, 99999]:
                    sub_output_array = np.zeros(24)
                    for hour in range(24):
                        maskindex = stack[:, hour]
                        mask = (maskindex >= min_lv) & (maskindex < max_lv)
                        filtered_lv_roads = df_roads.loc[mask]
                    #if len(filtered_lv_roads) == 0:
                    # output_array += [0.0]
                        sub_output_array[int(hour)] = mli.get_road_length_between_distances(receiver_point, min_dist, max_dist, filtered_lv_roads)
                    output_array = np.c_[output_array, sub_output_array]
                    min_lv = max_lv
                min_dist = max_dist
            print("  Calculate road length: ", time.time() - start2)
            input_data = np.vstack([input_data, output_array])
            print('  Total time: ', time.time() - start1)
            output_geom.append([i, Point(x_point, y_point)])
    i += 1
    if np.floor(i/20)*20 == i:
        np.save(file_features_save, input_data)
        np.save(file_geom_save, output_geom)

print('Done!')

C:\Users\dbotteld\.conda\envs\ml_model\lib\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


C:\Users\dbotteld\AppData\Local\Temp\9\ipykernel_11164\2058565192.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_roadsEB['lanes'][i] = min(list(map(int, eval(df_roadsEB['lanes'][i]))))


READING POINT (0/5855)
  Load roads:  0.0937507152557373
  Load buildings:  2.073060989379883
  Calculate traffic intensity:  1.3319313526153564
(24, 432)
  Calculate close objects:  30.80051612854004
  Calculate road length:  2.2394862174987793
  Total time:  38.02623891830444
READING POINT (1/5855)
  Load roads:  0.0937492847442627
  Load buildings:  2.9813570976257324
  Calculate traffic intensity:  1.4352607727050781
(24, 432)
  Calculate close objects:  29.36592721939087
  Calculate road length:  2.7222015857696533
  Total time:  37.93325662612915
READING POINT (2/5855)
  Load roads:  0.04687786102294922
  No roads
READING POINT (3/5855)
  Point in building (moving point...)
  Load roads:  0.0312497615814209
  No roads
READING POINT (4/5855)
  Load roads:  0.02602672576904297
  No roads
READING POINT (5/5855)
  Load roads:  0.062497615814208984
  Load buildings:  1.1900889873504639
  Calculate traffic intensity:  0.3249552249908447
(24, 432)
  Calculate close objects:  29.78104639

<__array_function__ internals>:5: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
C:\Users\dbotteld\.conda\envs\ml_model\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


  Load roads:  0.0468747615814209
  No roads
READING POINT (21/5855)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Load roads:  0.031264543533325195
  No roads
READING POINT (22/5855)
  Load roads:  0.09374856948852539
  Load buildings:  2.957064628601074
  Calculate traffic intensity:  1.3752319812774658
(24, 432)
  Calculate close objects:  29.109375
  Calculate road length:  2.9662983417510986
  Total time:  37.70311880111694
READING POINT (23/5855)
  Load roads:  0.06249856948852539
  Load buildings:  1.8071603775024414
  Calculate traffic intensity:  0.9188122749328613
  No close roads
(24, 432)
  Calculate close objects:  0.9344372749328613
  Calculate road length:  2.0129213333129883
  Total time:  5.5868353843688965
READING POINT (24/5855)
  Load roads:  0.0312497615814209
  No roads
READING POINT

...

<__array_function__ internals>:5: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
C:\Users\dbotteld\.conda\envs\ml_model\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


READING POINT (3720/5855)
  Load roads:  0.03125
  No roads
READING POINT (3721/5855)
  Load roads:  0.03125
  No roads
READING POINT (3722/5855)
  Load roads:  0.078125
  Load buildings:  1.2781901359558105
  Calculate traffic intensity:  0.6144320964813232
(24, 432)
  Calculate close objects:  26.670214891433716
  Calculate road length:  1.3527989387512207
  Total time:  31.094082355499268
READING POINT (3723/5855)
  Load roads:  0.046875
  Load buildings:  2.35268235206604
  Calculate traffic intensity:  0.976280689239502
(24, 432)
  Calculate close objects:  27.23438811302185
  Calculate road length:  1.7584316730499268
  Total time:  33.34805083274841
READING POINT (3724/5855)
  Load roads:  0.0468754768371582
  Load buildings:  1.3973045349121094
  Calculate traffic intensity:  0.30407190322875977
  No close roads
(24, 432)
  Calculate close objects:  0.31969714164733887
  Calculate road length:  1.5538320541381836
  Total time:  4.137890815734863
READING POINT (3725/5855)
  Load

<__array_function__ internals>:5: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
C:\Users\dbotteld\.conda\envs\ml_model\lib\site-packages\numpy\lib\npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


READING POINT (3740/5855)
  Load roads:  0.0468754768371582
  No roads
READING POINT (3741/5855)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Load roads:  0.031249284744262695
  No roads
READING POINT (3742/5855)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Point in building (moving point...)
  Load roads:  0.031249046325683

KeyboardInterrupt: 

In [18]:
#del GroupScaler
#from GScaler import GroupScaler

input_data = np.array(input_data)
output_geom = np.array(output_geom)
print(input_data.shape)
print(output_geom.shape)
scaler_input = pickle.load(open(file_input_scaler,'rb'))
input_data_scaled = scaler_input.transform(input_data)

data_length = len(input_data_scaled)
imagedata = np.zeros([data_length,22,36])
for y in range(data_length):
    for j in range(22):
        i = j
        if j%2 == 1:
            i = 1
        elif j == 0:
            i = 0
        else:
            i = int(j/2)+1
        imagedata[y][j]=input_data_scaled[y][0+(i*36):36+(i*36)]

distdata = np.zeros([data_length,20])
for p in range(data_length):
    distdata[p] = input_data_scaled[p][432:452]

output_data_scaled = model.predict([imagedata, distdata])
output_data = scaler_output.inverse_transform(output_data_scaled)
print(output_data.shape)

output_data = output_data.reshape((int(len(output_data)/24)), 24, 17)
print(output_data.shape)

np.save(file_indicators_save, output_data)
np.save(file_geom_save, output_geom)

(55392, 452)
(2308,)
1731/1731 [==============================] - 7s 4ms/step
(55392, 17)
(2308, 24, 17)


In [3]:
pyproj.datadir.set_data_dir('C:\\Users\\botteldo\\anaconda3\\envs\\ml_model\\Library\\share\\proj')


DataDirError: Valid PROJ data directory not found. Either set the path using the environmental variable PROJ_LIB or with `pyproj.datadir.set_data_dir`.

In [5]:
input_data.size


14297664

In [6]:
input_data.shape


(31632, 452)

In [17]:
class GroupScaler:
    import numpy as np
    def __init__(self):
        self.inputscalers = []
    
    def fit(self, samples): 
        Nsamples = samples.shape[0]
        samples_reordered = []
        #building densities here
        samples_reordered.append(np.reshape(samples[:,0:36],(36*Nsamples,)))
        #distances within 200m
        samples_reordered.append(np.reshape(samples[:,36:432],(((432-36)*Nsamples,))))
        #building density
        samples_reordered.append(samples[:,432])
        #road length
        samples_reordered.append(np.reshape(samples[:,433:437],(((437-433)*Nsamples,))))
        #building density
        samples_reordered.append(samples[:,437])
        #road length
        samples_reordered.append(np.reshape(samples[:,438:442],(((442-438)*Nsamples,))))
        #building density
        samples_reordered.append(samples[:,442])
        #road length
        samples_reordered.append(np.reshape(samples[:,443:447],(((447-443)*Nsamples,))))
        #building density
        samples_reordered.append(samples[:,447])
        #road length
        samples_reordered.append(np.reshape(samples[:,448:452],(((452-448)*Nsamples,))))

        # now create separate scalers for each group
        for input_d in range(0,len(samples_reordered)): 
            self.inputscalers.append(StandardScaler(with_mean=False))
            self.inputscalers[input_d].fit(samples_reordered[input_d].reshape(-1,1))
    
    def transform(self, samples):
        Nsamples = samples.shape[0]
        samples_reordered = []
        #building densities here
        samples_reordered.append(np.reshape(samples[:,0:36],(36*Nsamples,)))
        #distances within 200m
        samples_reordered.append(np.reshape(samples[:,36:432],(((432-36)*Nsamples,))))
        #building density
        samples_reordered.append(samples[:,432])
        #road length
        samples_reordered.append(np.reshape(samples[:,433:437],(((437-433)*Nsamples,))))
        #building density
        samples_reordered.append(samples[:,437])
        #road length
        samples_reordered.append(np.reshape(samples[:,438:442],(((442-438)*Nsamples,))))
        #building density
        samples_reordered.append(samples[:,442])
        #road length
        samples_reordered.append(np.reshape(samples[:,443:447],(((447-443)*Nsamples,))))
        #building density
        samples_reordered.append(samples[:,447])
        #road length
        samples_reordered.append(np.reshape(samples[:,448:452],(((452-448)*Nsamples,))))

        # now apply separate scalers for each group
        scaled_reordered = []
        for input_d in range(0,len(samples_reordered)): 
            scaled_reordered.append(self.inputscalers[input_d].transform(samples_reordered[input_d].reshape(-1,1)))
        
        # restore original sample
        scaled = np.zeros((Nsamples,452))
        #building densities
        scaled[:,0:36] = np.reshape(scaled_reordered[0],(Nsamples,36))
        #distances within 200m
        scaled[:,36:432] = np.reshape(scaled_reordered[1],(Nsamples,(432-36)))
        #building density
        scaled[:,432] = scaled_reordered[2].reshape(-1,)
        #road length
        scaled[:,433:437] = np.reshape(scaled_reordered[3],(Nsamples,(437-433)))
        #building density
        scaled[:,437] = scaled_reordered[4].reshape(-1,)
        #road length
        scaled[:,438:442] = np.reshape(scaled_reordered[5],(Nsamples,(442-438)))
        #building density
        scaled[:,442] = scaled_reordered[6].reshape(-1,)
        #road length
        scaled[:,443:447] = np.reshape(scaled_reordered[7],(Nsamples,(447-443)))
        #building density
        scaled[:,447] = scaled_reordered[8].reshape(-1,)
        #road length
        scaled[:,448:452] = np.reshape(scaled_reordered[9],(Nsamples,(452-448)))
        
        return scaled
    
    def inverse_transform(self, samples):
        Nsamples = samples.shape[0]
        samples_reordered = []
        #building densities
        samples_reordered.append(np.reshape(samples[:,0:36],(36*Nsamples,)))
        #distances within 200m
        samples_reordered.append(np.reshape(samples[:,36:432],(((432-36)*Nsamples,))))
        #building density
        samples_reordered.append(samples[:,432])
        #road length
        samples_reordered.append(np.reshape(samples[:,433:437],(((437-433)*Nsamples,))))
        #building density
        samples_reordered.append(samples[:,437])
        #road length
        samples_reordered.append(np.reshape(samples[:,438:442],(((442-438)*Nsamples,))))
        #building density
        samples_reordered.append(samples[:,442])
        #road length
        samples_reordered.append(np.reshape(samples[:,443:447],(((447-443)*Nsamples,))))
        #building density
        samples_reordered.append(samples[:,447])
        #road length
        samples_reordered.append(np.reshape(samples[:,448:452],(((452-448)*Nsamples,))))

        # now apply separate scalers for each group
        scaled_reordered = []
        for input_d in range(0,len(samples_reordered)): 
            scaled_reordered.append(self.inputscalers[input_d].inverse_transform(samples_reordered[input_d].reshape(-1,1)))
        
        # restore original sample
        scaled = np.zeros((Nsamples,452))
        #building densities
        scaled[:,0:36] = np.reshape(scaled_reordered[0],(Nsamples,36))
        #distances within 200m
        scaled[:,36:432] = np.reshape(scaled_reordered[1],(Nsamples,(432-36)))
        #building density
        scaled[:,432] = scaled_reordered[2].reshape(-1,)
        #road length
        scaled[:,433:437] = np.reshape(scaled_reordered[3],(Nsamples,(437-433)))
        #building density
        scaled[:,437] = scaled_reordered[4].reshape(-1,)
        #road length
        scaled[:,438:442] = np.reshape(scaled_reordered[5],(Nsamples,(442-438)))
        #building density
        scaled[:,442] = scaled_reordered[6].reshape(-1,)
        #road length
        scaled[:,443:447] = np.reshape(scaled_reordered[7],(Nsamples,(447-443)))
        #building density
        scaled[:,447] = scaled_reordered[8].reshape(-1,)
        #road length
        scaled[:,448:452] = np.reshape(scaled_reordered[9],(Nsamples,(452-448)))
        
        return scaled


In [14]:
i=20
np.floor(i/20)*20 ==i

True